In [2]:
import pandas as pd
import numpy as np
import sklearn

In [3]:
data = pd.read_csv("data.csv")
print(data.head())

        Name FrontShoes  RearShoes  AmericanStyleCart           CoachName  \
0   Loikkaus  HAS_SHOES  HAS_SHOES              False       Henna Latvala   
1  R.R. Samu  HAS_SHOES  HAS_SHOES              False         Erkki Jämsä   
2   Ekopetus  HAS_SHOES  HAS_SHOES              False        Antti Isomaa   
3     Huuger  HAS_SHOES  HAS_SHOES              False      Satu Siikamäki   
4     Wauhti  HAS_SHOES  HAS_SHOES              False  Veli-Erkki Paavola   

           DriverName  Distance  StartTrack        Sire      StartTime  \
0      Hannu Hietanen      2100           1      Apassi  1514823300000   
1          Mika Forss      2100           2  Suikun Ero  1514823300000   
2       Ville Pohjola      2100           3    A.T. Eko  1514823300000   
3         Niko Jokela      2100           4     Viesker  1514823300000   
4  Veli-Erkki Paavola      2100           5    Valtraus  1514823300000   

   StartType  Day  Month  Year  Winner   Odds  
0  CAR_START    1      1  2018   False   7.8

In [4]:
data.Winner = data.Winner.astype(int)
data.AmericanStyleCart = data.AmericanStyleCart.astype(int)
data.FrontShoes = data.FrontShoes.replace(to_replace=['NO_SHOES', 'HAS_SHOES', 'UNKNOWN'], value=[0, 1, -1])
data.RearShoes = data.RearShoes.replace(to_replace=['NO_SHOES', 'HAS_SHOES', 'UNKNOWN'], value=[0, 1, -1])
data.StartType = data.StartType.replace(to_replace=['CAR_START', 'VOLT_START'], value=[0, 1])


In [11]:
coach_df = pd.read_csv('coaches.csv')
driver_df = pd.read_csv('drivers.csv')

driver_df.columns = ['name', 'count',  'wins',  'winsPerRace',   'meanOdds',  'oddsTotal']
coach_df.columns = ['name', 'count',  'wins',  'winsPerRace',   'meanOdds',  'oddsTotal']

#print(driver_df.head())

x = driver_df.loc[driver_df['name'] == 'Ulf Ohlsson']
print(x.winsPerRace)

0    0.154888
Name: winsPerRace, dtype: float64


In [15]:

coachWprList = []
driverWprList = []

coaches = {}
drivers = {}

for i, row in data.iterrows():
    coachName = row['CoachName']
    driverName = row['DriverName']
    
    if driverName not in drivers:
        driverWpr = (driver_df.loc[driver_df['name'] == driverName]).winsPerRace
        drivers.update({driverName: driverWpr})
    else:
        driverWpr = drivers[driverName]
    
    if coachName not in coaches:
        coachWpr = (coach_df.loc[coach_df['name'] == coachName]).winsPerRace
        coaches.update({coachName: coachWpr})
    else:
        coachWpr = coaches[coachName] 
        
    driverWprList.append(driverWpr)
    coachWprList.append(coachWpr)



In [42]:
print(driverWprList[1].values[0])

0.1473609635139922


In [43]:
data['driverWpr'] = [driverWprList[i].values[0] for i in range(len(driverWprList))]
data['coachWpr'] = [coachWprList[i].values[0] for i in range(len(coachWprList))]

In [46]:
data.head()


,Name,FrontShoes,RearShoes,AmericanStyleCart,CoachName,DriverName,Distance,StartTrack,Sire,StartTime,StartType,Day,Month,Year,Winner,Odds,driverWpr,coachWpr
0,Loikkaus,1,1,0,Henna Latvala,Hannu Hietanen,2100,1,Apassi,1514823300000,0,1,1,2018,0,7.85,0.128536,0.048780
1,R.R. Samu,1,1,0,Erkki Jämsä,Mika Forss,2100,2,Suikun Ero,1514823300000,0,1,1,2018,0,3.88,0.147361,0.019608
2,Ekopetus,1,1,0,Antti Isomaa,Ville Pohjola,2100,3,A.T. Eko,1514823300000,0,1,1,2018,0,28.20,0.110599,0.049296
3,Huuger,1,1,0,Satu Siikamäki,Niko Jokela,2100,4,Viesker,1514823300000,0,1,1,2018,0,8.21,0.111872,0.071429
4,Wauhti,1,1,0,Veli-Erkki Paavola,Veli-Erkki Paavola,2100,5,Valtraus,1514823300000,0,1,1,2018,0,2.45,0.122905,0.230392


In [78]:
X = data.copy()

columns_to_remove = ['Sire', 'CoachName', 'DriverName', 'StartTime', 'Year', 'Day', 'Name', 'Winner']

for column in columns_to_remove:
    del X[column]
y = data.Winner

In [92]:
X.head()
X.to_csv('X.csv')

In [80]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [82]:
model = RandomForestRegressor(n_estimators=100, max_depth=4, random_state=0)
model.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [83]:
predictions = model.predict(X_test)

In [84]:
print(y_test.values[0])

0


In [91]:
x = 0
for  i in range(len(predictions)):
    if (predictions[i] > 0.6):
        
        print('Actual: ' + str(y_test.values[i]) + ' Predicted: ' + str(predictions[i]))
        #print(X_test.iloc[i:])
        x = x + 1

Actual: 0 Predicted: 0.6315240266519361
Actual: 0 Predicted: 0.6108512188189966
Actual: 0 Predicted: 0.6183057865094927
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6183057865094927
Actual: 1 Predicted: 0.6146938814648861
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 0 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 0 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 0 Predicted: 0.6264031283646019
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 0 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6207641001339573
Actual: 1 Predicted: 0.6258887276549582
Actual: 1 Predicted: 0.6315240266519361


Actual: 0 Predicted: 0.6108512188189966
Actual: 0 Predicted: 0.6108512188189966
Actual: 1 Predicted: 0.6287127065578819
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6287127065578819
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6157049642617803
Actual: 0 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6108512188189966
Actual: 1 Predicted: 0.6315240266519361
Actual: 0 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6289232044042237
Actual: 0 Predicted: 0.6207641001339573
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 0 Predicted: 0.6287127065578819
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6315240266519361
Actual: 1 Predicted: 0.6258887276549582


In [87]:
print(model.score(X_test, y_test))

0.18489194055359326
